In [3]:
from os import sysconf
import numpy as np
import pandas as pd
import random
import sys
import os

# Importing the dataset and removing the portugese songs
path = 'lyrics-data.csv'

dataset = pd.read_csv(path)
#print(dataset.head())

print(dataset['ALink'].unique())

dataset_eng = dataset[dataset['Idiom']=='ENGLISH']

print(dataset_eng['ALink'].unique())


['/10000-maniacs/' '/12-stones/' '/3-doors-down/' ... '/wilson-das-neves/'
 '/ze-ricardo/' '/zeca-pagodinho/']
['/10000-maniacs/' '/12-stones/' '/3-doors-down/' ... '/sambo/'
 '/seu-jorge/' '/zeca-pagodinho/']


In [14]:
# Process the data

lyrics_text = dataset_eng['Lyric'].str.cat(sep='.').lower()
# Print first 300 chars
print(lyrics_text[:300])

print('corpus length: ', len(lyrics_text))

chars_to_keep = ['\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
for c in lyrics_text:
    

chars = sorted(list(set(lyrics_text)))
print(chars)
print('total chars: ', len(chars))

i could feel at the time. there was no way of knowing. fallen leaves in the night. who can say where they're blowing. as free as the wind. hopefully learning. why the sea on the tide. has no way of turning. more than this. you know there's nothing. more than this. tell me one thing. more than this. 
corpus length:  167891278


NameError: name 'char' is not defined